<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-14 10:19:44
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: -2.17 K (-0.02%)
Current PnL: -21.88 L (-14.88%)
CY Booked + Current PnL: -13.61 L (-9.26%)
-------------------
Total profit:  2.54 L
Total loss:  -24.42 L
-------------------
Total Booked + Current PnL: 13.37 L (11.14%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.22%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.36 L (61.17%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.57%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.03,112080.0,7768.0,8877.0,5.01,7.45,7.92,15.96,11.0,0.88,0.86,25.60,X5K,ATH,METALS
68,SBIN,760.30,863.00,-11.34,M-LC,3.83,219062.0,17582.0,9639.0,0.58,8.73,4.40,13.51,61.0,1.82,1.68,22.42,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-14.30,H-LC,5.91,171237.0,15060.0,11028.0,-0.32,9.64,6.44,16.71,19.0,1.37,1.31,28.60,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,11.51,H-MC,13.83,159187.0,32987.0,13197.0,1.23,26.14,8.29,36.59,111.0,2.50,1.22,41.93,XY24,NTT,HEALTHCARE
83,TTKPRESTIG,769.29,770.00,92.53,M-SC,10.34,82988.0,-17789.0,17884.0,-0.24,-17.65,21.55,0.09,243.0,-0.99,0.64,9.03,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1219.98,2270.00,-45.46,M-SC,4.32,92190.0,692.0,78057.0,-0.97,0.76,84.67,86.07,219.0,0.01,0.71,33.36,XR,NTT,DURABLES
61,RAJOOENG,102.50,143.33,-32.60,H-SC,4.61,102080.0,-420.0,41251.0,1.07,-0.41,40.41,39.83,116.0,-0.01,0.78,1.07,AR,ATH,MISC
25,DABUR,505.20,735.00,-9.78,H-MC,3.49,196502.0,-526.0,90155.0,0.07,-0.27,45.88,45.49,103.0,-0.01,1.51,12.74,XY24,BTT,FMCG
90,VOLTAS,1278.28,1918.49,-9.05,H-MC,7.72,190860.0,-882.0,96919.0,-0.29,-0.46,50.78,50.08,101.0,-0.01,1.46,6.31,XY25,ATH,AC
92,WIPRO,243.46,420.00,-13.34,M-LC,7.35,152086.0,1141.0,108316.0,1.51,0.76,71.22,72.51,54.0,0.01,1.17,6.82,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-20.12,H-LC,2.57,213350.0,8850.0,32557.0,-0.24,4.33,15.26,20.25,10.0,0.27,1.64,13.44,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.67,H-LC,3.15,170340.0,-31046.0,118659.0,0.19,-15.42,69.66,43.51,27.0,-0.26,1.31,18.57,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-16.62,H-LC,3.25,128267.0,-137159.0,194068.0,-0.05,-51.68,151.30,21.44,8.0,-0.71,0.98,3.07,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-6.36,H-LC,3.73,185526.0,-14784.0,74767.0,0.76,-7.38,40.30,29.94,17.0,-0.20,1.42,28.17,X200,ATH,IT
37,HINDUNILVR,2413.81,2723.0,-10.99,H-LC,5.81,280134.0,9787.0,24848.0,0.24,3.62,8.87,12.81,28.0,0.39,2.15,17.16,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.03,112080.0,7768.0,8877.0,5.01,7.45,7.92,15.96,11.0,0.88,0.86,25.60,X5K,ATH,METALS
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.73,60.62,XR,NTT,CHEMICALS
10,BAJAJHFL,122.26,152.00,-11.52,H-MC,5.28,186468.0,-14038.0,62821.0,4.73,-7.00,33.69,24.33,98.0,-0.22,1.43,7.53,X40N,BTT,FINANCE
29,EASEMYTRIP,18.12,26.40,-7.85,M-SC,24.82,88700.0,-88115.0,168911.0,2.60,-49.83,190.43,45.70,197.0,-0.52,0.68,3.18,XY24,NTT,TRAVEL
45,INFY,1461.46,2275.00,-20.67,H-LC,10.92,259282.0,-2319.0,147946.0,1.54,-0.89,57.06,55.67,4.0,-0.02,1.99,5.16,X40,BTT,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMPUS,294.86,393.00,-30.40,M-SC,2.15,145038.0,-22737.0,78582.0,-2.22,-13.55,54.18,33.28,208.0,-0.29,1.11,12.29,XY24,NTT,FOOTWEAR
13,BATAINDIA,1550.24,2096.00,-40.29,M-SC,7.08,89325.0,-39345.0,84644.0,-1.89,-30.58,94.76,35.20,209.0,-0.46,0.68,0.00,X40,NTT,FOOTWEAR
65,SAIL,130.64,228.00,33.12,M-MC,10.20,209585.0,-15377.0,183031.0,-1.89,-6.84,87.33,74.53,193.0,-0.08,1.61,21.69,XY24,BTT,STEEL
70,SHALBY,261.39,327.00,941.47,M-SC,6.88,141746.0,-39397.0,84863.0,-1.84,-21.75,59.87,25.10,232.0,-0.46,1.09,13.60,XY24,NTT,HEALTHCARE
3,ALKYLAMINE,2347.98,4546.37,-5.01,H-SC,15.53,88369.0,-12594.0,107121.0,-1.81,-12.47,121.22,93.63,149.0,-0.12,0.68,31.62,SR,ATH,CHEMICALS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,92.53,M-SC,10.34,82988.0,-17789.0,17884.0,-0.24,-17.65,21.55,0.09,243.0,-0.99,0.64,9.03,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,82.03,H-SC,20.54,115344.0,-27135.0,46657.0,-1.04,-19.04,40.45,13.70,163.0,-0.58,0.88,36.14,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.81,H-SC,3.10,114390.0,-29413.0,56165.0,-0.87,-20.45,49.10,18.60,148.0,-0.52,0.88,26.23,OX40N,NTT,CERAMICS
49,KANSAINER,299.63,340.0,-68.89,H-SC,2.77,214380.0,-55287.0,91626.0,0.73,-20.50,42.74,13.47,142.0,-0.60,1.64,8.43,XY24,NTT,PAINTS
43,INDIGOPNTS,1407.73,1408.0,108.15,M-SC,2.92,136636.0,-37923.0,37957.0,0.25,-21.72,27.78,0.02,221.0,-1.00,1.05,18.48,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.96,236691.0,-59243.0,153163.0,-0.06,-20.02,64.71,31.73,2.0,-0.39,1.81,1.05,X40,BTT,IT
45,INFY,1461.46,2275.00,-20.67,H-LC,10.92,259282.0,-2319.0,147946.0,1.54,-0.89,57.06,55.67,4.0,-0.02,1.99,5.16,X40,BTT,IT
89,VBL,492.64,671.64,-11.01,H-LC,10.14,270291.0,5251.0,91061.0,-1.70,1.98,33.69,36.33,5.0,0.06,2.07,15.54,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.91,H-LC,10.45,197106.0,-19413.0,112311.0,-0.65,-8.97,56.98,42.91,7.0,-0.17,1.51,3.23,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.62,H-LC,3.25,128267.0,-137159.0,194068.0,-0.05,-51.68,151.30,21.44,8.0,-0.71,0.98,3.07,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,11.51,H-MC,13.83,159187.0,32987.0,13197.0,1.23,26.14,8.29,36.59,111.0,2.5,1.22,41.93,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,11.51,H-MC,13.83,159187.0,32987.0,13197.0,1.23,26.14,8.29,36.59,111.0,2.50,1.22,41.93,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,7.87,H-SC,12.21,128344.0,3047.0,55958.0,-1.19,2.43,43.60,47.09,141.0,0.05,0.98,60.63,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,1.98,H-MC,6.96,175930.0,11260.0,56702.0,0.20,6.84,32.23,41.28,91.0,0.20,1.35,15.37,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-4.72,H-SC,3.88,108245.0,6241.0,37680.0,1.16,6.12,34.81,43.06,123.0,0.17,0.83,24.28,X40N,ATH,MISC
35,HEROMOTOCO,4311.81,5949.07,-10.13,H-MC,7.86,166282.0,15369.0,41936.0,-0.37,10.18,25.22,37.97,92.0,0.37,1.27,38.06,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-10.13,H-MC,7.86,166282.0,15369.0,41936.0,-0.37,10.18,25.22,37.97,92.0,0.37,1.27,38.06,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,11.99,135097.0,11761.0,122006.0,-0.15,9.54,90.31,108.45,119.0,0.10,1.04,34.02,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,7.87,H-SC,12.21,128344.0,3047.0,55958.0,-1.19,2.43,43.60,47.09,141.0,0.05,0.98,60.63,MH,ATH,POWER
92,WIPRO,243.46,420.00,-13.34,M-LC,7.35,152086.0,1141.0,108316.0,1.51,0.76,71.22,72.51,54.0,0.01,1.17,6.82,XR,NTT,IT
91,WHIRLPOOL,1219.98,2270.00,-45.46,M-SC,4.32,92190.0,692.0,78057.0,-0.97,0.76,84.67,86.07,219.0,0.01,0.71,33.36,XR,NTT,DURABLES


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.58,72806.0,-40743.0,80793.0,-0.47,-35.88,110.97,35.27,268.0,-0.50,0.56,78.20,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-5.01,H-SC,15.53,88369.0,-12594.0,107121.0,-1.81,-12.47,121.22,93.63,149.0,-0.12,0.68,31.62,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-18.70,H-SC,13.91,92310.0,-5670.0,27536.0,-0.02,-5.79,29.83,22.32,152.0,-0.21,0.71,33.88,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.73,60.62,XR,NTT,CHEMICALS
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.03,112080.0,7768.0,8877.0,5.01,7.45,7.92,15.96,11.0,0.88,0.86,25.60,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-10.13,H-MC,7.86,166282.0,15369.0,41936.0,-0.37,10.18,25.22,37.97,92.0,0.37,1.27,38.06,AR,ATH,AUTO
51,LALPATHLAB,2838.53,3545.00,-25.70,H-SC,10.10,221741.0,28721.0,19314.0,-1.05,14.88,8.71,24.89,134.0,1.49,1.70,41.70,X40N,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.58,72806.0,-40743.0,80793.0,-0.47,-35.88,110.97,35.27,268.0,-0.50,0.56,78.20,XR,NTT,HOTELS
70,SHALBY,261.39,327.00,941.47,M-SC,6.88,141746.0,-39397.0,84863.0,-1.84,-21.75,59.87,25.10,232.0,-0.46,1.09,13.60,XY24,NTT,HEALTHCARE
89,VBL,492.64,671.64,-11.01,H-LC,10.14,270291.0,5251.0,91061.0,-1.70,1.98,33.69,36.33,5.0,0.06,2.07,15.54,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.40
1,25,41.15
2,50,70.95


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.39
LC    34.33
MC    26.30
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.85
X40N     15.55
X40      13.37
XY25     10.69
XR        9.06
AR        8.43
OX40N     7.52
X5K       2.27
MH        1.74
X200      1.42
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.97
H-SC    24.58
H-MC    23.35
M-SC    13.33
M-LC     7.27
M-MC     2.61
L-SC     1.48
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.23,-11.07,46.78
IT,12.01,-19.00,78.44
FINANCE,8.31,-20.26,64.76
BANKS,7.64,-14.36,62.93
MISC,5.91,-19.31,87.71
PAINTS,5.65,-18.56,41.93
HEALTHCARE,5.32,5.39,18.50
ELECTRICAL,4.74,-13.73,62.98
AUTO,4.47,-15.73,66.08


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2872992.0,23
XR,1112054.0,13
AR,1109497.0,10
X40,827705.0,10
XY25,654408.0,8
X40N,578323.0,12
OX40N,478469.0,10
SR,197120.0,2
X5K,142185.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2808487.0,28
M-SC,1508562.0,17
H-LC,1348504.0,18
H-MC,1330297.0,18
M-LC,430619.0,5
M-MC,337417.0,2
L-SC,268724.0,3
L-MC,58351.0,1
L-LC,44670.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      940741.0      6
M-SC       XY24      876714.0      7
H-SC       AR        673074.0      5
           XR        521085.0      6
H-LC       X40       514386.0      5
H-MC       XY24      496721.0      5
M-MC       XY24      337417.0      2
H-LC       XY25      255019.0      3
           AR        233218.0      2
H-MC       X40       228675.0      4
           XY25      221271.0      2
H-LC       X40N      220194.0      5
H-SC       X40N      215406.0      4
           OX40N     205103.0      4
           SR        197120.0      2
M-LC       XY24      179356.0      2
L-SC       XR        178435.0      2
M-SC       AR        161269.0      2
H-MC       X40N      142723.0      3
M-SC       OX40N     134370.0      4
M-LC       X5K       133308.0      1
M-SC       XR        127756.0      2
           XY25      123809.0      1
H-MC       XR        118111.0      1
M-LC       XR        108316.0      1
L-SC       OX40N      90289.0      1
M-SC       X40        84644.0      1
H-LC       X200       74767.0      1
L-MC       XR         58351.0      1
H-SC       MH         55958.0      1
H-MC       OX40N      48707.0      1
L-LC       XY25       44670.0      1
H-LC       XY24       42043.0      1
H-MC       AR         41936.0      1
           MH         32153.0      1
M-LC       XY25        9639.0      1
H-LC       X5K         8877.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
